In [1]:
#библиотеки для парсинга
import requests as rq

from bs4 import BeautifulSoup

#библиотеки для предобработки
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.tokenize import sent_tokenize

#библиотеки для определения ключевых слов
from collections import Counter
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
# функция парсинг статьи и её заголовка
def GetArticle(url0):
  page0 = rq.get(url0)
  soup0 = BeautifulSoup(page0.text, features="html.parser")
  title = soup0.find_all('h1', {'class':"jeg_post_title"})
  text_list = soup0.find_all('p')
  text = []
  for i in text_list:
    text.append(i.text)
  final_text = ' '.join(text)
  return url0, title, final_text

In [3]:
#парсинг статьей с сайта "Ватаным Татарстан"
articles = []
z = 1
while z <= 10:
  url = 'https://vatantat.ru/category/bezne%d2%a3-proektlar/page/' + str(z)
  page = rq.get(url)
  soup = BeautifulSoup(page.text, features="html.parser")
  soup.find_all("a")
  titles = []
  for x in soup.find_all("a"): 
    titles.append(x.text)
  links = []
  for link in soup.find_all("a"):
    links.append(link.get('href')) 
  data_1 = []
  for i in range(len(links)):
    if i == 222 or i == 228 or i == 234 or i == 240 or i == 246 or i == 252 or i == 258 or i == 264 or i == 270 or i == 276:
      data_1.append(links[i])
  for link in data_1:
    new = GetArticle(link)
    articles.append(new)
  z += 1

In [4]:
#создание датафрейма
art_df = pd.DataFrame(articles, columns = ['ссылка', 'заголовок', 'текст'])
art_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ссылка     100 non-null    object
 1   заголовок  100 non-null    object
 2   текст      100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [5]:
art_df[0:20]

,ссылка,заголовок,текст
0,https://vatantat.ru/2023/02/104776/,[[Иске орлыкны «уяту» ЫСУЛЛАРЫ]],"Үзең әзерләгән яки кибеттән сатып алып, кайча..."
1,https://vatantat.ru/2023/02/104769/,[[Иммунитетны ныгытуның ИҢ ГАДИ ЫСУЛЫ]],"5–6 канәферне 1 стакан кайнар суга салып, бер..."
2,https://vatantat.ru/2023/02/104752/,[[Яңа Шигалидә яшәгән Фәләховларның туганнары:...,"Бүген иртәнге җиде тулып киткәч, Питрәч район..."
3,https://vatantat.ru/2023/02/104705/,[[Көрәшче әнисе: «Балаларны «тыйнаклык ул – ма...,"Зур үрләр яулап, танылу алгач та, кеше булып ..."
4,https://vatantat.ru/2023/02/104653/,[[Саклану артык булмас: COVID-19 белән авыру о...,Фото: kzn.ru Россиядә соңгы дүрт атна дәвамын...
5,https://vatantat.ru/2023/02/104536/,[[5 төрле тәмле соус ясау ЫСУЛЫ]],"Фото: vk.com Нинди ризыкка тотынсак та, хәзер..."
6,https://vatantat.ru/2023/02/104527/,[[Пульмонолог Рөстәм Насретдинов: «Пневмония б...,"Фото: vk.com Үпкә авырулары, аерым алганда пн..."
7,https://vatantat.ru/2023/02/104508/,[[Юлда бер ирне коткарып танылган Алинә Махиян...,Алинә Махияновага – 26 яшь кенә әле. Чаллыдаг...
8,https://vatantat.ru/2023/02/104462/,[[Суши-торт ясау ЫСУЛЫ]],Гөлназ Ганиева рецепты Кирәк: 250 г пешкән дө...
9,https://vatantat.ru/2023/02/104147/,[[Пенсиягә чыгу стажын ничек сатып алырга? Бел...,Киләсе елда пенсиягә чыгу яшенә җитәм. Тик хә...


In [7]:
#предобработка статей: убираем лишние символы из заголовков и текстов
for index, row in art_df.iterrows():
    art_df['текст'][index] = art_df['текст'][index].replace('© 2019 Copyright © Все права защищены «Ватаным Татарстан» газетасы һәм сайты язмаларын, фото, видеоларны күчереп бастырган очракта редакциядән рөхсәт алу мәҗбүри ☎222-09-70✉info@ © 2019 Copyright © Все права защищены', '')
    art_df['текст'][index] = art_df['текст'][index].replace('© 2019 Copyright © Все права защищены «Ватаным Татарстан» газетасы һәм сайты язмаларын, фото, видеоларны күчереп бастырган очракта редакциядән рөхсәт алу мәҗбүри ☎222-09-70✉info@vatantat © 2019 Copyright © Все права защищены', '')
    art_df['текст'][index] = art_df['текст'][index].replace('© 2019  © Все права защищены «Ватаным Татарстан» газетасы һәм сайты язмаларын, фото, видеоларны күчереп бастырган очракта редакциядән рөхсәт алу мәҗбүри ☎222-09-70✉@ © 2019  © Все права защищены', '')
    art_df['текст'][index] = art_df['текст'][index].replace('Ваш адрес email не будет опубликован. Комментарий  Имя  Email', '')
    art_df['текст'][index] = art_df['текст'][index].replace('\n', '')
    art_df['текст'][index] = art_df['текст'][index].replace('Сабан туен көтәсезме?  Нәтиҗәләр', '')
    art_df['текст'][index] = art_df['текст'][index].replace('Фото:', '')
    art_df['текст'][index] = art_df['текст'][index].replace('фото:', '')
    art_df['текст'][index] = art_df['текст'][index].replace('ФОТО:', '')
    art_df['текст'][index] = art_df['текст'][index].replace('*', '')
    art_df['текст'][index] = art_df['текст'][index].replace('\xa0', '')
    art_df['текст'][index] = art_df['текст'][index].replace('.ru', '')
    art_df['текст'][index] = re.sub(r"[A-Za-z]+\.[A-Za-z]+", '', art_df['текст'][index])
    art_df['текст'][index] = re.sub(r"\n\s*\r", '', art_df['текст'][index])
    art_df['текст'][index] = re.sub(r"[A-z]+", '', art_df['текст'][index])

for index, row in art_df.iterrows():
    art_df['заголовок'][index] = str(art_df['заголовок'][index])
    art_df['заголовок'][index] = art_df['заголовок'][index].replace('[<h1 class="jeg_post_title">', '')
    art_df['заголовок'][index] = art_df['заголовок'][index].replace('</h1>]', '')
    
art_df[0:20]

,ссылка,заголовок,текст
0,https://vatantat.ru/2023/02/104776/,Иске орлыкны «уяту» ЫСУЛЛАРЫ,"Үзең әзерләгән яки кибеттән сатып алып, кайча..."
1,https://vatantat.ru/2023/02/104769/,Иммунитетны ныгытуның ИҢ ГАДИ ЫСУЛЫ,"5–6 канәферне 1 стакан кайнар суга салып, бер..."
2,https://vatantat.ru/2023/02/104752/,Яңа Шигалидә яшәгән Фәләховларның туганнары: “...,"Бүген иртәнге җиде тулып киткәч, Питрәч район..."
3,https://vatantat.ru/2023/02/104705/,Көрәшче әнисе: «Балаларны «тыйнаклык ул – мату...,"Зур үрләр яулап, танылу алгач та, кеше булып ..."
4,https://vatantat.ru/2023/02/104653/,Саклану артык булмас: COVID-19 белән авыру очр...,Россиядә соңгы дүрт атна дәвамында -19 белә...
5,https://vatantat.ru/2023/02/104536/,5 төрле тәмле соус ясау ЫСУЛЫ,"Нинди ризыкка тотынсак та, хәзер соус кулла..."
6,https://vatantat.ru/2023/02/104527/,Пульмонолог Рөстәм Насретдинов: «Пневмония бер...,"Үпкә авырулары, аерым алганда пневмония, үл..."
7,https://vatantat.ru/2023/02/104508/,Юлда бер ирне коткарып танылган Алинә Махиянов...,Алинә Махияновага – 26 яшь кенә әле. Чаллыдаг...
8,https://vatantat.ru/2023/02/104462/,Суши-торт ясау ЫСУЛЫ,Гөлназ Ганиева рецепты Кирәк: 250 г пешкән дө...
9,https://vatantat.ru/2023/02/104147/,Пенсиягә чыгу стажын ничек сатып алырга? Белге...,Киләсе елда пенсиягә чыгу яшенә җитәм. Тик хә...


In [8]:
#ищу и удаляю строки с пустыми ячейками "текст"

for index, row in art_df.iterrows():
    if art_df['текст'][index].strip() == '':
        art_df.drop(labels = [index], axis = 0, inplace = True)

art_df[0:20]

,ссылка,заголовок,текст
0,https://vatantat.ru/2023/02/104776/,Иске орлыкны «уяту» ЫСУЛЛАРЫ,"Үзең әзерләгән яки кибеттән сатып алып, кайча..."
1,https://vatantat.ru/2023/02/104769/,Иммунитетны ныгытуның ИҢ ГАДИ ЫСУЛЫ,"5–6 канәферне 1 стакан кайнар суга салып, бер..."
2,https://vatantat.ru/2023/02/104752/,Яңа Шигалидә яшәгән Фәләховларның туганнары: “...,"Бүген иртәнге җиде тулып киткәч, Питрәч район..."
3,https://vatantat.ru/2023/02/104705/,Көрәшче әнисе: «Балаларны «тыйнаклык ул – мату...,"Зур үрләр яулап, танылу алгач та, кеше булып ..."
4,https://vatantat.ru/2023/02/104653/,Саклану артык булмас: COVID-19 белән авыру очр...,Россиядә соңгы дүрт атна дәвамында -19 белә...
5,https://vatantat.ru/2023/02/104536/,5 төрле тәмле соус ясау ЫСУЛЫ,"Нинди ризыкка тотынсак та, хәзер соус кулла..."
6,https://vatantat.ru/2023/02/104527/,Пульмонолог Рөстәм Насретдинов: «Пневмония бер...,"Үпкә авырулары, аерым алганда пневмония, үл..."
7,https://vatantat.ru/2023/02/104508/,Юлда бер ирне коткарып танылган Алинә Махиянов...,Алинә Махияновага – 26 яшь кенә әле. Чаллыдаг...
8,https://vatantat.ru/2023/02/104462/,Суши-торт ясау ЫСУЛЫ,Гөлназ Ганиева рецепты Кирәк: 250 г пешкән дө...
9,https://vatantat.ru/2023/02/104147/,Пенсиягә чыгу стажын ничек сатып алырга? Белге...,Киләсе елда пенсиягә чыгу яшенә җитәм. Тик хә...


In [10]:
#ищу и удаляю интервью
        
for index, row in art_df.iterrows():
    a = art_df['заголовок'][index]
    if re.search(r'\w+: «', a) != None or re.search(r'\w+: “', a) != None: 
        art_df.drop(labels = [index], axis = 0, inplace = True)
art_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 98
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ссылка     81 non-null     object
 1   заголовок  81 non-null     object
 2   текст      81 non-null     object
dtypes: object(3)
memory usage: 4.6+ KB


In [11]:
#упорядочить индексы
art_df.index = np.arange(len(art_df))

In [12]:
#столбец для суммаризированных статей
art_df['суммаризация'] = ""

In [15]:
#ищу рецепты и копирую в столбец "суммаризация"
for index, row in art_df.iterrows():
    if re.search(r'\d+ г ', art_df['текст'][index]) != None or re.search(r'\d+ гр ', art_df['текст'][index]) != None or re.search(r'\d+ кг ', art_df['текст'][index]) != None or 'ЫСУЛЫ' in art_df['заголовок'][index] or 'ЫСУЛЛАРЫ' in art_df['заголовок'][index]:
        art_df['суммаризация'][index] = art_df['текст'][index]

In [16]:
#перемещаемся в папку библиотеки py_tat_morphan
import os
os.chdir('/Users/Zila/opt/anaconda3/lib/python3.9/site-packages/py_tat_morphan')
from py_tat_morphan.morphan import Morphan
morphan = Morphan()
print(morphan.lemma('урманнарга'))

['урман']


In [17]:
#список всех слов из словаря библиотеки
words = []
with open('py_tat_morphan/files/words.csv', 'r') as f:
    for i in f:
        t = i.split()
        words.append(t[0])

In [18]:
#cписок стоп-слов
tatar_stopwords = ['кеше', 'кирәк', 'шуңа', 'күрә','ул', 'бару', 'кылу', 'һәм', 'белән', 'да', 'дә', 'бу', 'ул', 'дип', 'өчен', 'бер', 'иде', 'аның', 'буенча', 'ә', 'генә', 'бик', 'инде', 'шул', 'бар', 'гына', 'шулай', 'үз', 'ук', 'турында', 'алар', 'булган', 'булып', 'итеп', 'торган', 'әле', 'итә', 'дигән', 'аны', 'була', 'икән', 'бит', 'кебек', 'булды', 'кадәр', 'менә', 'ди', 'тагын', 'тә', 'килә', 'арасында', 'тора', 'килеп', 'тиеш', 'әмма', 'анда', 'диде', 'та', 'аларның', 'итү', 'килгән', 'һәр', 'үзе', 'шушы', 'исә', 'ала', 'мөмкин', 'ләкин', 'тик', 'аңа', 'шуңа', 'итте', 'бөтен', 'чөнки', 'нинди', 'ни', 'ничек', 'туры', 'бара', 'артык', 'төп']

stop_lemm = []
for i in range(len(tatar_stopwords)):
    word_lemm = morphan.lemma(tatar_stopwords[i])
    stop_lemm = stop_lemm + word_lemm
    
stop_lemm = list(set(stop_lemm))

print(stop_lemm)

['кадәр', 'тагын', 'ә', 'бик', 'күрә', 'өчен', 'ләкин', 'ул', 'чөнки', 'дә', 'тә', 'шул', 'бер', 'тор', 'кыл', 'әле', 'мөмкин', 'үз', 'һәм', 'и', 'күр', 'инде', 'анда', 'һәр', 'белән', 'нинди', 'кеше', 'ис', 'ти', 'кебек', 'та', 'шушы', 'тур', 'тиеш', 'ни', 'икән', 'гына', 'ук', 'туры', 'ал', 'да', 'кил', 'әмма', 'төп', 'үзе', 'кирәк', 'ничек', 'буенча', 'шулай', 'артык', 'ара', 'мен', 'ит', 'турында', 'бар', 'бит', 'арасында', 'тик', 'алар', 'генә', 'бу', 'бул', 'исә', 'менә', 'ала', 'бөтен', 'ди']


In [19]:
#лемматизация статей
art_df['lemmas'] = ''
for index, row in art_df.iterrows():
    text = art_df['текст'][index]
    text = re.sub(r"^\s+|\s+$", "", text)
    
    text_low = text.lower()
        
    text_list_nltk = word_tokenize(text_low)
        
    text_without_punkt = [] 
    for word in text_list_nltk:
        if word[0].isalpha(): 
            text_without_punkt.append(word)
                
    tokens_lem = []
    tokens_except = []
    for i in text_without_punkt:
        if i in words:
            tokens_lem.append(morphan.lemma(i)[0])
        else:
            tokens_except.append(i)
                
    text_clean = []
    for word in tokens_lem:
        if word not in stop_lemm:
            text_clean.append(word)
    text_clean = ' '.join(text_clean)
    
    art_df['lemmas'][index] = text_clean
art_df[0:10]

,ссылка,заголовок,текст,суммаризация,lemmas
0,https://vatantat.ru/2023/02/104776/,Иске орлыкны «уяту» ЫСУЛЛАРЫ,"Үзең әзерләгән яки кибеттән сатып алып, кайча...","Үзең әзерләгән яки кибеттән сатып алып, кайча...",әзерлә яки кибет сат кайчангы оныт иске орлык ...
1,https://vatantat.ru/2023/02/104769/,Иммунитетны ныгытуның ИҢ ГАДИ ЫСУЛЫ,"5–6 канәферне 1 стакан кайнар суга салып, бер...","5–6 канәферне 1 стакан кайнар суга салып, бер...",канәфер стакан кайна су сал сәгать төнә көн дә...
2,https://vatantat.ru/2023/02/104653/,Саклану артык булмас: COVID-19 белән авыру очр...,Россиядә соңгы дүрт атна дәвамында -19 белә...,,россия соңгы дүрт атна дәвам чирлә очрак ар кү...
3,https://vatantat.ru/2023/02/104536/,5 төрле тәмле соус ясау ЫСУЛЫ,"Нинди ризыкка тотынсак та, хәзер соус кулла...","Нинди ризыкка тотынсак та, хәзер соус кулла...",ризык тот хәзер соус куллан киң тара ризык эст...
4,https://vatantat.ru/2023/02/104462/,Суши-торт ясау ЫСУЛЫ,Гөлназ Ганиева рецепты Кирәк: 250 г пешкән дө...,Гөлназ Ганиева рецепты Кирәк: 250 г пешкән дө...,ганиева пеш дөге тоз кызыл балык эремчек сыр к...
5,https://vatantat.ru/2023/02/104147/,Пенсиягә чыгу стажын ничек сатып алырга? Белге...,Киләсе елда пенсиягә чыгу яшенә җитәм. Тик хә...,,ел пенсия чыгу яшь җит хәзер аз ял чыгу стаж п...
6,https://vatantat.ru/2023/02/104140/,«Файдасы юк»: кайсы очракларда каһвәдән бөтенл...,"Арыганда көч өсти, күңелне күтәрә. Ул булганд...",,ары көч өстә күңел күтәр бернинди дару түгел з...
7,https://vatantat.ru/2023/02/104118/,Расписка кәгазе судта юридик көчкә ияме?,Хезмәттәшем бер ел элек бурычка зур суммада а...,,хезмәттәш ел элек бурыч зур сумма акча бурыч е...
8,https://vatantat.ru/2023/02/104083/,Табиблар чаң суга: Татарстанда яман шеш белән ...,Узган ел Татарстанда 17049 кешедә яман шеш ...,,уз ел татарстан яма шеш ачыкла ел чагыштыр очр...
9,https://vatantat.ru/2023/02/104065/,Мич коймагы пешерү ЫСУЛЫ,Ярты литр сөт һәм җылымса суга 5 йомырка сы...,Ярты литр сөт һәм җылымса суга 5 йомырка сы...,ярты литр сөт су йомырка сыт аш кашык тоз аш к...


In [20]:
#столбец с самыми частотными 5 словами
art_df['common'] = ''
for index, row in art_df.iterrows():
    tok = word_tokenize(art_df['lemmas'][index])
    art_df['common'][index] = Counter(tok).most_common(5)

In [21]:
#столбец с самыми частотными 5 словами TF-IDF
art_df['tfidf'] = ''
tfidf = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1, 1))
articles_tfidf = tfidf.fit_transform(art_df['lemmas'])

def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

feature_names = np.array(tfidf.get_feature_names_out())

for i, article in enumerate(art_df['lemmas']):
    article_vector = articles_tfidf[i]
    words_tdidf = get_top_tf_idf_words(article_vector, feature_names, 5)
    art_df['tfidf'][i] = list(words_tdidf)

In [22]:
#сравнение частотных слов td-idf и в рамках одного текста
art_td = art_df[['tfidf', 'common']]
art_td[0:10]

,tfidf,common
0,"[орлык, сәгать, тот, сал, уят]","[(орлык, 23), (сәгать, 12), (сал, 11), (тот, 1..."
1,"[канәфер, кулъяулык, лаванда, чәйнә, файда]","[(канәфер, 2), (файда, 2), (яки, 2), (май, 2),..."
2,"[йок, очрак, сана, мәскәү, прививка]","[(очрак, 5), (авыр, 4), (йок, 4), (сана, 4), (..."
3,"[соус, сыр, бәрәңге, тоз, борыч]","[(соус, 6), (тоз, 5), (борыч, 4), (бәрәңге, 4)..."
4,"[дөге, өс, эремчек, сыр, тез]","[(өс, 6), (дөге, 4), (эремчек, 3), (сыр, 3), (..."
5,"[пенсия, ел, стаж, чыгу, сат]","[(ел, 15), (пенсия, 9), (чыгу, 6), (җит, 5), (..."
6,"[эч, эчемлек, файда, кофеин, бас]","[(эч, 14), (файда, 6), (эчемлек, 5), (кан, 5),..."
7,"[расписка, бурыч, кәгазь, кайтар, заем]","[(бурыч, 5), (кайтар, 4), (расписка, 4), (кәга..."
8,"[яма, шеш, республика, кушымта, ачыкла]","[(яма, 16), (шеш, 14), (республика, 7), (ел, 6..."
9,"[грамм, кашык, калак, тугла, күпер]","[(аш, 2), (кашык, 2), (грамм, 2), (ярты, 1), (..."


In [24]:
#суммаризация
for index, row in art_df.iterrows():
    if art_df['суммаризация'][index] == '':
        text = art_df['текст'][index]
        text = re.sub(r"^\s+|\s+$", "", text)
        td_words = art_df['tfidf'][index]
        
        sentences = sent_tokenize(text)
        summarized = []
        summarized.append(''.join(sentences[0]))
        
        questions = []
        for sentence in sentences[1:len(sentences)-3]:
            if sentence[len(sentence)-1] == '?':
                questions.append(sentence)
    
            else:
                tokenized = word_tokenize(sentence)
                tokenized_lemm = []
                tokens_except = []
                for word in tokenized:
                    if word in words:
                        tokenized_lemm.append(morphan.lemma(word)[0])
                    else:
                        tokens_except.append(i)
                
                for word in tokenized_lemm:
                    for i in range(len(td_words)):
                        if word == td_words[i]:
                            if sentence not in summarized:
                                if sentences[sentences.index(sentence)][0:9] == 'Шуңа күрә':
                                    sentence = sentences[sentences.index(sentence)][10:]
                                    sentence = sentence.capitalize()
                                    summarized.append(''.join(sentence))
                                elif sentences[sentences.index(sentence)][0:4] == 'Әмма':
                                    sentence = sentences[sentences.index(sentence)][5:]
                                    sentence = sentence.capitalize()
                                    summarized.append(''.join(sentence))
                                elif sentences[sentences.index(sentence)][0:5] == 'Ләкин':
                                    sentence = sentences[sentences.index(sentence)][6:]
                                    sentence = sentence.capitalize()
                                    summarized.append(''.join(sentence))
                                else:    
                                    summarized.append(''.join(sentence))
                

        summarized.append(sentences[len(sentences) - 2])  
        
        art_df['суммаризация'][index] = summarized
        

In [25]:
#перееводим суммаризированные тексты из списков в строки
for index, row in art_df.iterrows():
    if type(art_df['суммаризация'][index]) == list:
        art_df['суммаризация'][index] = ' '.join(art_df['суммаризация'][index])
    art_df['суммаризация'][index] = re.sub(r"\. –", ". ", art_df['суммаризация'][index])

In [26]:
art_df['дата'] = ""

In [27]:
#узнать текущую дату
import datetime
from datetime import date
date = str(date.today())
tod = date.split('-')

In [28]:
#расставить даты для публикации к текстам
for index, row in art_df.iterrows():
    if index > 0:
        art_df['дата'][index] = date
        date_spl = date.split('-')
        day = int(date_spl[2])
        month = int(date_spl[1])
        year = int(date_spl[0])
        if day == 31 and month == 12:
            day = 1
            month = 1
            year = year + 1
        elif day == 31 and (month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10 or month == 12):
            day = 1
            month = month + 1
        elif day == 30 and (month == 4 or month == 6 or month == 9 or month == 11):
            day = 1
            month = month + 1
        elif day == 28 and month == 2:
            day = 1
            month = month + 1
        else:
            day += 1
        date = str(year) + '-' + str(month) + '-' + str(day)

In [29]:
#расставить даты по мусульманскому календарю
from hijri_converter import Hijri, Gregorian
art_df['мус_дата'] = ""
for index, row in art_df.iterrows():
    if index > 0:
        date = art_df['дата'][index]
        date_spl = date.split('-')
        day = int(date_spl[2])
        month = int(date_spl[1])
        year = int(date_spl[0])
        hijri = Gregorian(year, month, day).to_hijri()
        art_df['мус_дата'][index] = hijri

In [30]:
#если есть, соответствующие датам праздники
art_df['праздники'] = ''
for index, row in art_df.iterrows():
    if index > 0:
        if '2023-2-17' in art_df['дата'][index]:
            art_df['праздники'][index] = '26нан 27нче Раҗәбкә – Мигъраҗ кичәсе'
        elif '2023-2-21' in art_df['дата'][index]:
            art_df['праздники'][index] ='1 нче Шәгъбан'
        elif '2023-3-6' in art_df['дата'][index]:
            art_df['праздники'][index] = '14нән 15нче Шәгъбангә – Бәраәт кичәсе'
        elif '2023-3-21' in art_df['дата'][index]:
            art_df['праздники'][index] = 'Нәүрүз бәйрәме'
        elif '2023-3-23' in art_df['дата'][index]:
            art_df['праздники'][index] = '1 нче Рамазан – Рамазан ае башлану'
        elif '2023-4-17' in art_df['дата'][index]:
            art_df['праздники'][index] = '26нан 27нче Рамазанга – Кадер кичәсе'
        elif '2023-4-21' in art_df['дата'][index]:
            art_df['праздники'][index] = '1 нче Шәүвәл – Ураза гаете'
        else:
            art_df['праздники'][index] = ' '

In [31]:
#сохраняем в файл
art_df.to_csv('art.csv')

In [32]:
#открываем файл
with open("art.csv", "r") as file:
    art = pd.read_csv(file, delimiter=',')


In [33]:
monthsdict = {
    '1':"гыйнвар", 
    '2':"февраль",
    '3':"март",
    '4':"апрель",
    '5':"май",
    '6':"июнь",
    '7':"июль",
    '8':"август",
    '9':"сентябрь",
    '10':"октябрь",
    '11':"ноябрь",
    '12':"декабрь"
}

In [34]:
hijramonths = {
    '1':"Мөхәррәм", 
    '2':"Сәфәр",
    '3':"Рабигыль-әүвәл",
    '4':"Рабигыль-ахыр",
    '5':"Җөмадиәл-әүвәл",
    '6':"Җөмадиәл-ахыр",
    '7':"Рәҗәп",
    '8':"Шәгъбан",
    '9':"Рамазан",
    '10':"Шәүвәл",
    '11':"Зөлкагдә",
    '12':"Зөлхиҗҗә"
}

In [35]:
#текст для публикации
for index, row in art.iterrows():
    if index > 0:
        date = art['дата'][index]
        date_spl = str(date).split('-')
        day = date_spl[2]
        month = int(date_spl[1])
        year = date_spl[0]

        if tod[0] in year and month == int(tod[1]) and tod[2] in day:
            tatmonth = monthsdict.get(str(month))
            day1 = str()
            if (len(day) == 1 and (int(day) == 1 or int(day) == 2 or int(day) == 3 or int(day) == 4 or int(day) == 5 or int(day) == 7 or int(day) == 8)) or (len(day) == 2 and (int(day[1]) == 1 or int(day[1]) == 2 or int(day[1]) == 3 or int(day[1]) == 4 or int(day[1]) == 5 or int(day[1]) == 7 or int(day[1]) == 8)):
                day1 = day + '-нче'
            elif (len(day) == 1 and (int(day) == 6 or int(day) == 9)) or (len(day) == 2 and (int(day[1]) == 6 or int(day[1]) == 9 or int(day[1]) == 0)):
                day1 = day + '-нчы'

            hijradate = art['мус_дата'][index]
            hdate_spl = hijradate.split('-')
            hday = hdate_spl[2]
            hmonth = int(hdate_spl[1])
            hyear = hdate_spl[0]
            hijramonth = hijramonths.get(str(hmonth))
            hday_ord = str()
            if (len(hday) == 1 and (int(hday) == 1 or int(hday) == 2 or int(hday) == 3 or int(hday) == 4 or int(hday) == 5 or int(hday) == 7 or int(hday) == 8)) or (len(hday) == 2 and (int(hday[1]) == 1 or int(hday[1]) == 2 or int(hday[1]) == 3 or int(hday[1]) == 4 or int(hday[1]) == 5 or int(hday[1]) == 7 or int(hday[1]) == 8)):
                hday_ord = hday + '-нче'
            elif (len(hday) == 1 and (int(hday) == 6 or int(hday) == 9)) or (len(hday) == 2 and (int(hday[1]) == 6 or int(hday[1]) == 9 or int(hday[1]) == 0)):
                hday_ord = hday + '-нчы'
            print('\033[1m' + 'Бүген', year + '-нче елның', day1, tatmonth, 'көне' +',', hijramonth, 'аеның', hday_ord, 'көне' + '. \n ')
            print(art['праздники'][index])
            print('\033[0m' + art['заголовок'][index], '\n ')

            text = art['суммаризация'][index]
            text1 = ''.join(text)
            print(text1, '\n ')
            print('Тулы мәкаләне "Ватаным Татарстан" сайтында укый аласыз. Сылтама:', art['ссылка'][index])


Бүген 2023-нче елның 13-нче февраль көне, Рәҗәп аеның 22-нче көне. 
 
 
Иммунитетны ныгытуның ИҢ ГАДИ ЫСУЛЫ 
 
 5–6 канәферне 1 стакан кайнар суга салып, бер сәгать төнәтеп, көн дәвамында җылы килеш эчәргә кирәк. Канәферне вакыт-вакыт чәйнәп йөрү дә файдалы. Ул йоткылык өстенә кергән вирусларны юк итәргә булыша. Лаванда, яшел чәй, әрем яки ак чыршы (пихта) эфир майлары сеңдерелгән кулъяулыкны иснәү дә файдалы. Янәшәгездәге кеше төчкерсә яки ютәлләсә бигрәк тә. Эш өстәленә берничә тамчы эвкалипт мае тамызу да зыян итмәс.                     
 
Тулы мәкаләне "Ватаным Татарстан" сайтында укый аласыз. Сылтама: https://vatantat.ru/2023/02/104769/
